- write a function to split data
- run all ML stuff func def
- run on 1-gram
- run on 2-gram
- run on ngram
- tfid 
- how to use word2vec?
- fast text

In [1]:
import pandas as pd
import swifter
import numpy as np
import nltk
import re

In [2]:
data = pd.read_csv('final_processed_df.csv')

In [3]:
data.head()

,qid,question_text,target,length,capitals,caps_vs_length,num_words,num_unique_words,words_vs_unique,num_exclamation_marks,num_question_marks,num_punctuation,num_symbols,processed_text,len_after,num_words_after,len_ratio,num_words_ratio
0,00002165364db923c7e6,How did Quebec nationalists see their province...,0,72,2,0.027778,13,13,1.0000,0,1,0,0,quebec nationalist see province nation,38,5,1.894737,2.600000
1,000032939017120e6e44,"Do you have an adopted dog, how would you enco...",0,81,1,0.012346,16,15,0.9375,0,1,1,0,adopt dog would encourage people adopt not shop,47,8,1.723404,2.000000
2,0000412ca6e4628ce2cf,Why does velocity affect time? Does velocity a...,0,67,2,0.029851,10,8,0.8000,0,2,0,0,velocity affect time velocity affect space geo...,51,7,1.313725,1.428571
3,000042bf85aa498cd78e,How did Otto von Guericke used the Magdeburg h...,0,57,4,0.070175,9,9,1.0000,0,1,0,0,otto von guericke use magdeburg hemisphere,42,6,1.357143,1.500000
4,0000455dfa3e01eae3af,Can I convert montra helicon D to a mountain b...,0,77,3,0.038961,15,15,1.0000,0,1,0,0,convert montra helicon mountain bike change tyre,48,7,1.604167,2.142857


In [4]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1306122 entries, 0 to 1306121
Data columns (total 18 columns):
qid                      1306122 non-null object
question_text            1306122 non-null object
target                   1306122 non-null int64
length                   1306122 non-null int64
capitals                 1306122 non-null int64
caps_vs_length           1306122 non-null float64
num_words                1306122 non-null int64
num_unique_words         1306122 non-null int64
words_vs_unique          1306122 non-null float64
num_exclamation_marks    1306122 non-null int64
num_question_marks       1306122 non-null int64
num_punctuation          1306122 non-null int64
num_symbols              1306122 non-null int64
processed_text           1305749 non-null object
len_after                1306122 non-null int64
num_words_after          1306122 non-null int64
len_ratio                1306122 non-null float64
num_words_ratio          1306122 non-null float64
dtypes: floa

In [5]:
def float_to_int(x):
    if x == float('inf') or x == float('-inf'):
        return float(0) # or a large value you choose
    return int(x)

data.len_ratio = data.len_ratio.swifter.apply(float_to_int)
data.num_words_ratio = data.num_words_ratio.swifter.apply(float_to_int)

data.processed_text = data.processed_text.fillna('')

In [6]:
data.describe()

,target,length,capitals,caps_vs_length,num_words,num_unique_words,words_vs_unique,num_exclamation_marks,num_question_marks,num_punctuation,num_symbols,len_after,num_words_after,len_ratio,num_words_ratio
count,1.306122e+06,1.306122e+06,1.306122e+06,1.306122e+06,1.306122e+06,1.306122e+06,1.306122e+06,1.306122e+06,1.306122e+06,1.306122e+06,1.306122e+06,1.306122e+06,1.306122e+06,1.306122e+06,1.306122e+06
mean,6.187018e-02,7.067884e+01,2.860756e+00,4.557576e-02,1.280361e+01,1.213578e+01,9.666687e-01,1.742563e-03,1.057475e+00,2.904055e-01,1.543730e-02,4.271438e+01,6.453952e+00,1.206526e+00,1.660784e+00
std,2.409197e-01,3.878428e+01,2.573325e+00,3.919927e-02,7.052437e+00,6.040779e+00,5.687022e-02,4.676167e-02,2.588720e-01,7.637911e-01,1.502499e-01,2.517267e+01,3.730951e+00,5.222969e-01,6.736842e-01
min,0.000000e+00,1.000000e+00,0.000000e+00,0.000000e+00,1.000000e+00,1.000000e+00,9.523810e-02,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
25%,0.000000e+00,4.500000e+01,1.000000e+00,2.000000e-02,8.000000e+00,8.000000e+00,9.354839e-01,0.000000e+00,1.000000e+00,0.000000e+00,0.000000e+00,2.600000e+01,4.000000e+00,1.000000e+00,1.000000e+00
50%,0.000000e+00,6.000000e+01,2.000000e+00,3.389831e-02,1.100000e+01,1.100000e+01,1.000000e+00,0.000000e+00,1.000000e+00,0.000000e+00,0.000000e+00,3.600000e+01,5.000000e+00,1.000000e+00,2.000000e+00
75%,0.000000e+00,8.500000e+01,4.000000e+00,5.882353e-02,1.500000e+01,1.500000e+01,1.000000e+00,0.000000e+00,1.000000e+00,0.000000e+00,0.000000e+00,5.300000e+01,8.000000e+00,1.000000e+00,2.000000e+00
max,1.000000e+00,1.017000e+03,1.800000e+02,1.000000e+00,1.340000e+02,9.600000e+01,1.000000e+00,5.000000e+00,1.000000e+01,3.500000e+01,2.000000e+01,6.840000e+02,6.900000e+01,8.100000e+01,2.800000e+01


In [7]:
sincere_df = data[data['target']==0]
insincere_df = data[data['target']==1]

In [8]:
sincere_df_subset = sincere_df.sample(n=160000)

In [9]:
insincere_df = insincere_df.reset_index()
sincere_df_subset = sincere_df_subset.reset_index()

In [10]:
sincere_df_subset.shape

(160000, 19)

In [11]:
subset = pd.concat([sincere_df_subset, insincere_df],0)
subset = subset.drop(['index'],1)
subset.head()

,qid,question_text,target,length,capitals,caps_vs_length,num_words,num_unique_words,words_vs_unique,num_exclamation_marks,num_question_marks,num_punctuation,num_symbols,processed_text,len_after,num_words_after,len_ratio,num_words_ratio
0,789c14fc0d9daa84e32d,What is a good engagement present idea for bes...,0,56,1,0.017857,10,10,1.0,0,1,0,0,good engagement present idea good friend,40,6,1.0,1.0
1,9149d6e89eceaafb6ad7,How can I be a trend-setter?,0,28,2,0.071429,6,6,1.0,0,1,0,0,trendsetter,11,1,2.0,6.0
2,7afb39c6055b0d587eed,Do I need bagpipe covers and cords and drone v...,0,52,2,0.038462,10,9,0.9,0,1,0,0,need bagpipe cover cord drone valve,35,6,1.0,1.0
3,3180c1c2b0bffe83f40d,Why is this KLM plane named after an American ...,0,60,5,0.083333,11,11,1.0,0,1,0,0,klm plane name american national park,37,6,1.0,1.0
4,7fb0425314a6b7cd11fe,What are the main causes of musculoskeletal di...,0,61,5,0.081967,9,9,1.0,0,1,0,0,main cause musculoskeletal disorder msd,39,5,1.0,1.0


In [12]:
subset.shape

(240810, 18)

- bag of words (TF)
- TF-IDF
- word2vec
* fasttext

In [16]:
from sklearn.feature_extraction.text import CountVectorizer

cv = CountVectorizer(min_df=3, max_df=.95)
cv_matrix = cv.fit_transform(subset.processed_text)
cv_matrix = cv_matrix.toarray()
cv_matrix

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]], dtype=int64)

In [17]:
# get all unique words in the corpus
vocab = cv.get_feature_names()
# show document feature vectors
cv_df = pd.DataFrame(cv_matrix, columns=vocab)

In [18]:
cv_df.shape

(240810, 23490)

In [16]:
subset.head()

,qid,question_text,target,length,capitals,caps_vs_length,num_words,num_unique_words,words_vs_unique,num_exclamation_marks,num_question_marks,num_punctuation,num_symbols,processed_text,len_after,num_words_after,len_ratio,num_words_ratio
0,86182f2f48fdced522f7,What is the difference between substance abuse...,0,71,1,0.014085,10,9,0.900000,0,1,0,0,difference substance abuse substance addiction,46,5,1.0,2.0
1,4f02fcc23b07be0f440e,How was SpaceX made?,0,20,3,0.150000,4,4,1.000000,0,1,0,0,spacex make,11,2,1.0,2.0
2,67010d02365f4e3162a7,How strong is the Hindu caste system in India?,0,46,3,0.065217,9,9,1.000000,0,1,0,0,strong hindu caste system india,31,5,1.0,1.0
3,8862d4bbf6f9808ede6e,Where can you buy real Rolex watches?,0,37,2,0.054054,7,7,1.000000,0,1,0,0,buy real rolex watch,20,4,1.0,1.0
4,e772db093b385de1cc3e,Who was the Supreme British Commander in the M...,0,69,6,0.086957,11,10,0.909091,0,1,0,0,supreme british commander middle east,37,5,1.0,2.0


In [ ]:
test = pd.concat([cv_df, subset.length],1)

In [ ]:
subset = subset.loc[~subset.index.duplicated(keep='first')]
cv_df = cv_df.loc[~cv_df.index.duplicated(keep='first')]

In [ ]:
pd.concat([subset, cv_df], 1)

In [17]:
pd.concat([subset.length, subset.capitals, subset.caps_vs_length, subset.num_words, subset.words_vs_unique,
           subset.num_unique_words, subset.num_exclamation_marks, subset.num_question_marks,
           subset.num_punctuation, subset.num_symbols, subset.len_after, subset.num_words_after,
           subset.len_ratio, subset.num_words_ratio],1)

InvalidIndexError: Reindexing only valid with uniquely valued Index objects

In [39]:
bv = CountVectorizer(ngram_range=(2,2), min_df=1, max_df=.95)
bv_matrix = bv.fit_transform(subset.processed_text)

bv_matrix = bv_matrix.toarray()
vocab = bv.get_feature_names()
bv_df = pd.DataFrame(bv_matrix, columns=vocab)

In [40]:
bv_df.shape

(240810, 882477)

In [41]:
bv_df.head()

,aa aa,aa actually,aa airman,aa attack,aa baitho,aa battery,aa bb,aa boot,aa chapter,aa could,...,zyklon shocking,zymase directly,zypp opensuse,zz cyclic,zz purely,zz snore,zzz breast,zzzquil gel,zzzquil work,zzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzz hi
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [25]:
from sklearn.feature_extraction.text import TfidfVectorizer

tv = TfidfVectorizer(min_df=0.001, max_df=0.99, use_idf=True, ngram_range=(1,2))
tv_matrix = tv.fit_transform(subset.processed_text)
tv_matrix = tv_matrix.toarray()

vocab = tv.get_feature_names()
tfid_df = pd.DataFrame(np.round(tv_matrix, 2), columns=vocab)

In [26]:
tfid_df.shape

(240810, 1317)

In [27]:
tfid_df.head()

,able,abortion,abroad,abuse,accept,acceptable,access,accomplishment,accord,account,...,ww,year,year old,yearold,yes,yet,york,young,yous,youtube
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [31]:
cv_bv = CountVectorizer(ngram_range=(1,2), min_df=1, max_df=.95)
cv_bv_matrix = cv_bv.fit_transform(subset.processed_text)

cv_bv_matrix = cv_bv_matrix.toarray()
vocab = cv_bv.get_feature_names()
ngram = pd.DataFrame(cv_bv_matrix, columns=vocab)

In [33]:
ngram.head()

,aa,aa actually,aa battery,aa bb,aa chapter,aa corporate,aa eerily,aa engine,aa holder,aa mean,...,zyklon gas,zyklon poison,zyklon shocking,zz,zz frac,zz real,zzz,zzz breast,zzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzz,zzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzz hi
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [ ]:
ngram_df = pd.concat([subset, ngram], 1)
ngram_df.head()

/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  """Entry point for launching an IPython kernel.


In [38]:
def text_preprocess(text):
    text = text.lower() 
    text = re.sub('[^a-zA-Z]', ' ', text) 
    text = re.sub(r'\s+', ' ', text)
    return text

In [39]:
text_preprocess('ABCDE .   Kdjkajd*(@*9992183 jdshj .         OKDDHJKAHDJcjda8983987)')

'abcde kdjkajd jdshj okddhjkahdjcjda '

In [40]:
# Cleaing the text
q_text = subset.question_text.swifter.apply(text_preprocess)

Pandas Apply: 100%|██████████| 240810/240810 [00:04<00:00, 57309.68it/s]


In [48]:
q_text.head()

0    is sociopathy and dissociative identity disord...
1    if r and s are two equivalence relation on a t...
2    was midsomer murders tv show a satire on bigot...
3    is the conflict between israel and arabic coun...
4                  what is the definition of a number 
Name: question_text, dtype: object

In [49]:
# Preparing the dataset
#all_ques = nltk.sent_tokenize(q_text)

all_words = [nltk.word_tokenize(sent) for sent in q_text]

In [50]:
from gensim.models import Word2Vec

word2vec = Word2Vec(all_words, min_count=2)  

In [51]:
vocabulary = word2vec.wv.vocab

In [54]:
word2vec.wv.most_similar('toxic')

[('cisgender', 0.8012945055961609),
 ('submissive', 0.7903337478637695),
 ('insensitive', 0.7729671001434326),
 ('silky', 0.7701021432876587),
 ('patriarchal', 0.7670682072639465),
 ('judgmental', 0.7654932737350464),
 ('stubborn', 0.762769877910614),
 ('hairfall', 0.7620580792427063),
 ('egotistical', 0.7615426182746887),
 ('toned', 0.7573975920677185)]